In [ ]:
if True:
    %pip install pyserial

In [ ]:
import math
import CalcLidarData # local file

# Lidar code adapted from https://github.com/henjin0/LIDAR_LD06_python_loder
# Download and run that to see a pointcloud
# MIT license

Hardware setup:

Connect the lidar to your computer


In [ ]:

import serial

 
ser = serial.Serial(port='COM69', # TODO: change this to your com port or /dev/tty... or /dev/ACM
                    baudrate=230400,
                    timeout=5.0,
                    bytesize=8,
                    parity='N',
                    stopbits=1)


Add your callback, like printing or adding it to a list

In [ ]:
l = []
def the_callback(data): # First sample will be shorter
    # print(data)
    # data: list of [angle, dist], sorted by angle, starting with lowest
    # angle is in radians, clockwise seen from above
    l.append(data)

Run for some time, default is max amount of samples, but you can change it to anything you want, like time based, triggered by some value, ...

In [ ]:
max_samples = 100


angles = list()
distances = list()
i = 0
tmpString = ""
sample = 0
while sample < max_samples:
    all_b = ser.read_all()
    for b in all_b:
        tmpInt = int( b)
        b = bytes([b])
        if (tmpInt == 0x54):
            tmpString +=  b.hex()+" "
            flag2c = True
            continue
        
        elif(tmpInt == 0x2c and flag2c):
            tmpString += b.hex()
            if(not len(tmpString[0:-5].replace(' ','')) == 90 ):
                tmpString = ""
                loopFlag = False
                flag2c = False
                continue

            lidarData = CalcLidarData.CalcLidarData(tmpString[0:-5])
            angles.extend(lidarData.Angle_i)
            distances.extend(lidarData.Distance_i)
            if(len(angles) > 50*12):
                split = [i for i in range(len(angles)-1) if angles[i+1]< angles[i]]
                first = angles[:split[0]+1]
                angles = angles[split[0]+1:]
                firstDist = distances[:split[0]+1]
                distances = distances[split[0]+1:]
                the_callback(list(zip(first, firstDist)))
                sample+=1
            tmpString = ""
        else:
            tmpString += b.hex()+" "
        
        flag2c = False

ser.close()

Do some fancy stuff with the data:

In [ ]:
print(len(l), [len(i) for i in l])
# First sample will always be a bit shorter